In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

## Prioritas Scrapping

In [2]:
path_of_the_directory= '/work/Prioritas'

In [3]:
prioritas2024 = []
prioritas2019 = []
for filename in os.listdir(path_of_the_directory):
    fname = path_of_the_directory + '/' + filename
    f = open(fname, 'r')
    soup= BeautifulSoup(f.read(), 'html.parser')
    items = soup.find_all('tr', {'role':'row'})[1:-1]
    year = filename[:4]
    for item in items:
        name = item.find('a').text.strip()
        info = item.find_all('td')
        date = info[4].text.strip()
        sponsor = info[9].text.strip()
        status = info[5].text.strip()
        data = [name, date, sponsor, status, year]
        category = info[7].text
        if category == 'Prolegnas 2015-2019':
            prioritas2019.append(data)
        else:
            prioritas2024.append(data) 

In [4]:
df2019_prioritas_raw = pd.DataFrame(prioritas2019, columns=['Judul Ruu', 'Tanggal Diupdate', 'Pengusul', 'Status', 'Tahun Prioritas'])
df2019_prioritas_raw.head()

,Judul Ruu,Tanggal Diupdate,Pengusul,Status,Tahun Prioritas
0,RUU tentang Perubahan atas Undang-Undang Nomor...,10 Mar 2020,DPR,Harmonisasi,2015
1,RUU tentang Radio Televisi Republik Indonesia,07 Feb 2018,DPR,Penyusunan,2015
2,RUU tentang Perubahan atas Undang - Undang Nom...,08 Des 2016,PEMERINTAH,Selesai,2015
3,RUU tentang Wawasan Nusantara,24 Okt 2016,DPD,Pembahasan,2015
4,RUU tentang Pertanahan,09 Sep 2019,DPR,Pembahasan,2015


In [5]:
#remove duplicate
df2019_prioritas = df2019_prioritas_raw.copy()
df2019_prioritas = df2019_prioritas.sort_values('Tahun Prioritas', ascending=False)
df2019_prioritas = df2019_prioritas.drop_duplicates(subset='Judul Ruu', keep='first')
print(len(df2019_prioritas))

103


In [6]:
df2024_prioritas_raw = pd.DataFrame(prioritas2024, columns=['Judul Ruu', 'Tanggal Diupdate', 'Pengusul', 'Status', 'Tahun Prioritas'])
df2024_prioritas_raw

,Judul Ruu,Tanggal Diupdate,Pengusul,Status,Tahun Prioritas
0,RUUtentang Perubahan atas Undang-Undang Nomor ...,20 Jan 2021,DPR,Harmonisasi,2020
1,RUU tentang Kitab Undang-Undang Hukum Pidana,06 Jul 2022,"DPR, PEMERINTAH",Pembahasan,2020
2,RUU tentang Perubahan atas Undang-Undang Nomor...,07 Jul 2022,"DPR, PEMERINTAH",Pembahasan,2020
3,RUU tentang Jabatan Hakim,23 Agust 2022,DPR,Terdaftar,2020
4,RUU tentang Perubahan atas Undang-Undang Nomor...,07 Des 2021,"DPR, PEMERINTAH",Selesai,2020
...,...,...,...,...,...
109,RUU tentang Wabah,23 Agust 2022,PEMERINTAH,Terdaftar,2022
110,RUU tentang Badan Pembinaan Ideologi Pancasila,12 Mei 2020,PEMERINTAH,Penetapan Usul,2022
111,RUU tentang Desain Industri,23 Agust 2022,PEMERINTAH,Terdaftar,2022
112,RUU tentang Daerah Kepulauan,23 Agust 2022,DPD,Pembahasan,2022


In [7]:
#remove duplicate
df2024_prioritas = df2024_prioritas_raw.copy()
df2024_prioritas = df2024_prioritas.sort_values('Tahun Prioritas', ascending=False)
df2024_prioritas = df2024_prioritas.drop_duplicates(subset='Judul Ruu', keep='first')
print(len(df2024_prioritas))

66


In [8]:
prioritas2024_list = df2024_prioritas['Judul Ruu'].tolist()
prioritas2019_list = df2019_prioritas['Judul Ruu'].tolist()
print('RUU tentang Jabatan Hakim' in prioritas2024_list)

True


## Prolegnas Scrapping

In [9]:
def connectDPR(url):
    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    req = requests.get(url, headers=headers)
    print(req)
    return BeautifulSoup(req.text, 'html.parser')

### Collecting all specific links of prolegnas 

In [10]:
url24 = 'https://www.dpr.go.id/uu/prolegnas-long-list'
soup = connectDPR(url24)
head = soup.find('tbody')
items = head.find_all('tr', 'role' == 'row')

<Response [200]>


In [11]:
links2024 = []
links2019 = []
main = 'https://www.dpr.go.id'
for item in items:
    category = item.find_all('td')[7].text.strip()
    address = item.find('a', href=True)
    if category == 'Prolegnas 2015-2019':
        links2019.append(main + address['href'])
    else:
        links2024.append(main + address['href'])

In [12]:
print(len(links2024))
print(len(links2019))

256
190


### Create a dataframe of prolegnas information from specific link

In [13]:
def recent_status(status_img, sponsors):
    if 'DPR' in sponsors:
        if ('grey' not in status_img[6]['src']):
            status = 'Selesai'
        elif ('grey' not in status_img[5]['src']):
            status = 'Keputusan'
        elif ('grey' not in status_img[4]['src']):
            status = 'Pembahasan'
        elif ('grey' not in status_img[3]['src']):
            status = 'Penetapan Usul'
        elif ('grey' not in status_img[2]['src']):
            status = 'Harmonisasi'
        elif ('grey' not in status_img[1]['src']):
            status = 'Penyusunan'
        else:
            status = 'Terdaftar'
    if 'PEMERINTAH' in sponsors:
        if ('grey' not in status_img[3]['src']):
            status = 'Selesai'
        elif ('grey' not in status_img[2]['src']):
            status = 'Keputusan'
        elif ('grey' not in status_img[1]['src']):
            status = 'Pembahasan'
        else:
            status = 'Terdaftar'
    if sponsors == 'DPD':
        if ('grey' not in status_img[4]['src']):
            status = 'Selesai'
        elif ('grey' not in status_img[3]['src']):
            status = 'Keputusan'
        elif ('grey' not in status_img[2]['src']):
            status = 'Pembahasan'
        elif ('grey' not in status_img[1]['src']):
            status = 'Penetapan Usul'
        else:
            status = 'Terdaftar'
    return status

### Test

In [14]:
links2024[5]

'https://www.dpr.go.id/uu/detail/id/243'

In [15]:
soup = connectDPR('https://www.dpr.go.id/uu/detail/id/1')
head = soup.find('div', {'class':'card-body'})
pengusul = soup.select('#accordion > div:nth-child(2) > div > div:nth-child(1) > div > div.col-md-7 > div > div > ul')
penugasan = soup.select('#accordion > div:nth-child(2) > div > div:nth-child(1) > div > div.col-md-5 > div > div > ul')
penugasan = "|".join([i.text.strip() for i in penugasan[0]])
pengusul = "|".join([i.text.strip() for i in pengusul[0]])
name = head.find('h3', {'class':'title-detail'}).text.strip()
date = head.find('li').text[17:]
sponsor = head.find('span', {'class':'label label-dpr'}).text.strip()

status_img = head.findAll('img', {'class':'progres-icon'})
status = recent_status(status_img, sponsor)
print(penugasan)
print(name, date, sponsor, status)

<Response [200]>
|Komisi I (set_komisi1@dpr.go.id)|
RUU tentang Perubahan atas Undang-Undang Nomor 32 Tahun 2002 tentang Penyiaran 02 Februari 2015 DPR Keputusan


### Prolegnas 2024 via Specific Link

In [ ]:
all_data2024 = []

for link in links2024:
    soup = connectDPR(link)
    head = soup.find('div', {'class':'card-body'})

    name = head.find('h3', {'class':'title-detail'}).text.strip()
    date = head.find('li').text[17:]
    sponsor = head.find('span', {'class':'label label-dpr'}).text.strip()

    state_img = head.find_all('img', {'class':'progres-icon'})
    state = recent_status(state_img, sponsor)

    prioritas = 'Tidak'
    if name in prioritas2024_list:
            prioritas = 'Ya'
    
    data = [name, date, sponsor, state, prioritas]
    all_data2024.append(data)

<Response [200]>
<Response [200]>
<Response [200]>


In [ ]:
df2024_specific = pd.DataFrame(all_data2024, columns=['Judul Ruu', 'Tanggal Diusul', 'Pengusul', 'Status', 'Prioritas'])
df2024_specific.head()

,Judul Ruu,Tanggal Diusul,Pengusul,Status,Prioritas
0,RUU tentang Keamanan dan Ketahanan Siber,17 Desember 2019,DPR,Terdaftar,Tidak
1,RUU tentang Perubahan atas Undang-Undang Nomor...,17 Desember 2019,DPR,Penyusunan,Ya
2,RUU tentang Radio Televisi Republik Indonesia,17 Desember 2019,DPR,Terdaftar,Tidak
3,RUU tentang Keamanan Laut,17 Desember 2019,DPR,Terdaftar,Tidak
4,RUU tentang Perubahan atas Undang-Undang Nomor...,17 Desember 2019,DPR,Terdaftar,Tidak


### Prolegnas 2019 via Specific Link

In [ ]:
all_data2019 = []

for link in links2019:
    soup = connectDPR(link)
    head = soup.find('div', {'class':'card-body'})

    name = head.find('h3', {'class':'title-detail'}).text
    date = head.find('li').text[17:].strip()
    sponsor = head.find('span', {'class':'label label-dpr'}).text.strip()

    state_img = head.find_all('img', {'class':'progres-icon'})
    state = recent_status(state_img, sponsor)

    prioritas = 'Tidak'
    if name in prioritas2019_list:
            prioritas = 'Ya'
    
    data = [name, date, sponsor, state, prioritas]

    data = [name, date, sponsor, state]
    all_data2019.append(data)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [ ]:
df2019_specific = pd.DataFrame(all_data2019, columns=['Judul Ruu', 'Tanggal Diusul', 'Pengusul', 'Status', 'Prioritas'])
df2019_specific.head()

ValueError: 5 columns passed, passed data had 4 columns

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=45b28f99-d503-49a3-bd9e-f98821f9fde8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>